In [1]:
import pandas as pd 
from tqdm import tqdm
tqdm.pandas()

In [2]:
df = pd.read_csv("../data/data.csv")

In [3]:
df.head()

,text,intent
0,It'll be great if you could delete this class ...,delete_class_in_curr_file
1,please can you close this file called shells.py,close_file
2,Could you please rename method named hello tha...,rename_fun_in_curr_file
3,Would you mind copying passenger function from...,copy_fun_in_another_file
4,Would you please modify the name of routine na...,rename_fun_in_another_file


In [4]:
df.describe()

,text,intent
count,73343,73343
unique,72019,27
top,Would you mind running this code,delete_class_in_curr_file
freq,3,3720


In [5]:
df["intent"].unique()

array(['delete_class_in_curr_file', 'close_file',
       'rename_fun_in_curr_file', 'copy_fun_in_another_file',
       'rename_fun_in_another_file', 'rename_class_in_curr_file',
       'delete_fun_in_another_file', 'rename_file',
       'copy_fun_in_curr_file', 'undo', 'open_file',
       'delete_class_in_another_file', 'delete_fun_in_curr_file',
       'import_fun', 'rename_class_in_another_file',
       'move_class_in_curr_file', 'import_class',
       'move_class_in_another_file', 'move_fun_in_curr_file', 'save_file',
       'delete_file', 'move_fun_in_another_file',
       'copy_class_in_curr_file', 'create_file',
       'copy_class_in_another_file', 'redo', 'compile'], dtype=object)

In [6]:
df['intent'].value_counts()

intent
delete_class_in_curr_file       3720
copy_class_in_curr_file         3720
copy_fun_in_curr_file           3717
delete_fun_in_curr_file         3655
import_fun                      3509
delete_class_in_another_file    3132
move_class_in_curr_file         3131
move_fun_in_curr_file           3129
delete_fun_in_another_file      3128
rename_class_in_curr_file       3060
move_class_in_another_file      3059
rename_fun_in_curr_file         3058
rename_fun_in_another_file      3056
rename_class_in_another_file    3056
move_fun_in_another_file        3027
rename_file                     3002
import_class                    2969
save_file                       2158
open_file                       2158
delete_file                     2154
close_file                      2150
copy_class_in_another_file      2040
copy_fun_in_another_file        2038
create_file                     1736
redo                            1283
undo                            1265
compile                        

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [14]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens
    

In [10]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\user\Documents\Personal\chatbot-with-intent-classification\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\Documents\Personal\chatbot-with-intent-classification\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either nee

In [11]:
texts = df["text"].tolist()
embeddings = model.encode(
    texts,
    batch_size=128,               # CPU optimal
    convert_to_numpy=True,
    show_progress_bar=True
)

df["sbert_encoded"] = embeddings.tolist()

Batches: 100%|██████████| 573/573 [04:08<00:00,  2.30it/s]


In [16]:
df["tokenized_text"] = df['text'].progress_apply(preprocess)

100%|██████████| 73343/73343 [02:01<00:00, 601.36it/s]


In [17]:
df.to_csv("../data/cleaned_data.csv", index=False)  